In [2]:
import os
import pandas as pd
from datetime import datetime
import pickle
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

def load_data(parent_folder, last_updated):
    data = []
    subfolders = sorted([f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))])

    for subfolder in subfolders:
        subfolder_path = os.path.join(parent_folder, subfolder)
        csv_files = sorted([f for f in os.listdir(subfolder_path) if f.endswith('.csv') and datetime.strptime(f.split('_')[-1].split('.')[0], '%Y%m%d') > last_updated])

        for csv_file in csv_files:
            file_path = os.path.join(subfolder_path, csv_file)
            encoding = detect_encoding(file_path)
            temp_df = pd.read_csv(file_path, encoding=encoding)
            data.append(temp_df)

    return pd.concat(data, ignore_index=True)

def save_pickle(df, file_path):
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)

def load_pickle(file_path):
    with open(file_path, 'rb') as f:
        return pickle.load(f)

parent_folder = 'data'
pickle_file = 'data.pickle'

if os.path.exists(pickle_file):
    df = load_pickle(pickle_file)
    last_updated = df['日時'].max()
    last_updated = datetime.strptime(str(last_updated), '%Y%m%d%H%M')
else:
    df = pd.DataFrame()
    last_updated = datetime.strptime('19700101', '%Y%m%d')

new_data = load_data(parent_folder, last_updated)

if not new_data.empty:
    df = pd.concat([df, new_data], ignore_index=True)
    save_pickle(df, pickle_file)

print(df.head())


NameError: name 'df' is not defined